In [16]:
pip install yfinance --quiet

Note: you may need to restart the kernel to use updated packages.


In [17]:
import yfinance as yf
import numpy as np
import pandas as pd

In [18]:
# Define the tickers for the indices and equities
index_tickers = ["^GSPC", "^FTSE", "^N225", "^GDAXI", "^HSI"]
equity_tickers = ["AAPL", "MSFT", "AMZN", "GOOGL","TSLA"]

# Index         Ticker Symbol    Equity   Ticker Symbol
# S&P 500         ^GSPC          Apple         AAPL
# DAX PERFORMANCE ^GDAXI         Microsoft     MSFT
# Hang Seng       ^HSI           Amazon        AMZN
# FTSE 100        ^FTSE          Google        GOOG
# Nikkei 225      ^N225          TESLA         TSLA

In [19]:
# Download historical data since "2010-01-01" for indices
index_data = yf.download(index_tickers, start="2010-01-01")["Adj Close"]

# Download historical data since "2010-01-01" for equities
equity_data = yf.download(equity_tickers, start="2010-01-01")["Adj Close"]

[*********************100%***********************]  5 of 5 completed
[*********************100%***********************]  5 of 5 completed


In [20]:
# Combine index and equity data into a single DataFrame
data = pd.concat([index_data, equity_data], axis=1)

## Daily returns->  Daily returns in finance refer to the percentage change in the price of a financial asset, such as a stock or an index, from one trading day to the next.

## Daily Return = ((Price at time t / Price at time t-1) - 1)*100

In [21]:
# Filter data to include common date range
start_date = max(data.index.min(), pd.to_datetime("2010-01-01"))
end_date = min(data.index.max(), pd.to_datetime("2023-05-17"))
data = data.loc[start_date:end_date]


In [22]:
# Calculate daily returns
returns = data.pct_change().dropna()

In [23]:

# Calculate cumulative returns
cumulative_returns = (returns + 1).cumprod()

In [24]:
# Calculate max drawdowns
rolling_max = cumulative_returns.expanding().max()
drawdown = (cumulative_returns / rolling_max) - 1
max_drawdown = drawdown.min()

In [25]:
# Calculate annualized volatility
volatility = returns.std() * np.sqrt(252)  # Assuming 252 trading days in a year

In [26]:
# Calculate the risk-free rate (considering it as 0% for simplicity)
risk_free_rate = 0.2

In [27]:
# Calculate Sharpe ratio
sharpe_ratio = (returns.mean() - risk_free_rate) / volatility

In [28]:
# Calculate downside returns for Sortino ratio
downside_returns = returns.copy()
downside_returns[downside_returns > 0] = 0

In [29]:
# Calculate downside volatility for Sortino ratio
downside_volatility = downside_returns.std() * np.sqrt(252)

In [30]:
# Calculate Sortino ratio
sortino_ratio = (returns.mean() - risk_free_rate) / downside_volatility

In [31]:
# Create a DataFrame for daily returns
daily_returns_df = pd.DataFrame(returns)
daily_returns_df.columns = index_tickers + equity_tickers

In [32]:
# Create a DataFrame for cumulative returns
cumulative_returns_df = pd.DataFrame(cumulative_returns.iloc[-1])
cumulative_returns_df.columns = ["Cumulative Returns"]


In [33]:
# Create a DataFrame for max drawdowns
max_drawdown_df = pd.DataFrame(max_drawdown)
max_drawdown_df.columns = ["Max Drawdown"]

In [34]:
# Create a DataFrame for volatility
volatility_df = pd.DataFrame(volatility)
volatility_df.columns = ["Volatility"]

In [35]:

# Create a DataFrame for Sharpe ratio
sharpe_ratio_df = pd.DataFrame(sharpe_ratio)
sharpe_ratio_df.columns = ["Sharpe Ratio"]

In [36]:
# Create a DataFrame for Sortino ratio
sortino_ratio_df = pd.DataFrame(sortino_ratio)
sortino_ratio_df.columns = ["Sortino Ratio"]

In [37]:
# Display the results
print("Daily Returns - Indices:")
print(daily_returns_df[index_tickers].to_string())

print("\nDaily Returns - Equities:")
print(daily_returns_df[equity_tickers].to_string())

Daily Returns - Indices:
               ^GSPC     ^FTSE     ^N225        ^GDAXI      ^HSI
Date                                                            
2010-06-30  0.000549  0.002266 -0.010113 -5.921811e-03 -0.019647
2010-07-01 -0.022596 -0.018119 -0.003240  0.000000e+00 -0.020361
2010-07-02  0.006721 -0.003974 -0.004662 -1.111183e-02  0.001318
2010-07-05 -0.003018 -0.003077  0.000000 -3.171066e-03  0.006853
2010-07-06  0.029336  0.021454  0.005359  1.219219e-02  0.007690
2010-07-07  0.010030  0.008733  0.031331 -1.130489e-02 -0.006253
2010-07-08  0.018087  0.007142  0.009413  9.744148e-03  0.027597
2010-07-09  0.005367  0.004901  0.007204  1.636361e-02  0.005199
2010-07-12  0.006643  0.001970  0.000733  4.356004e-03 -0.003882
2010-07-13  0.020128  0.018749  0.015379 -1.776928e-03 -0.001139
2010-07-14 -0.003320  0.003009 -0.000155  6.350625e-03  0.027053
2010-07-15 -0.008033 -0.009727  0.001196 -1.484335e-02 -0.011200
2010-07-16 -0.010055 -0.017740 -0.028819 -2.695047e-04 -0.028617


In [38]:
print("\nCumulative Returns:")
print(cumulative_returns_df.to_string())

print("\nMax Drawdowns:")
print(max_drawdown_df.to_string())

print("\nVolatility:")
print(volatility_df.to_string())

print("\nSharpe Ratio:")
print(sharpe_ratio_df.to_string())

print("\nSortino Ratio:")
print(sortino_ratio_df.to_string())



Cumulative Returns:
        Cumulative Returns
^FTSE             1.571080
^GDAXI            2.678117
^GSPC             3.947121
^HSI              0.966007
^N225             3.144356
AAPL             22.128157
AMZN             20.882055
GOOGL            10.512965
MSFT             17.361127
TSLA            104.554189

Max Drawdowns:
        Max Drawdown
^FTSE      -0.366055
^GDAXI     -0.387794
^GSPC      -0.339250
^HSI       -0.557008
^N225      -0.317989
AAPL       -0.437972
AMZN       -0.561453
GOOGL      -0.443201
MSFT       -0.371485
TSLA       -0.736322

Volatility:
        Volatility
^FTSE     0.158766
^GDAXI    0.199441
^GSPC     0.173401
^HSI      0.196369
^N225     0.201193
AAPL      0.279835
AMZN      0.325200
GOOGL     0.268512
MSFT      0.257404
TSLA      0.563532

Sharpe Ratio:
        Sharpe Ratio
^FTSE      -1.258549
^GDAXI     -1.000934
^GSPC      -1.150689
^HSI       -1.018154
^N225      -0.991969
AAPL       -0.710850
AMZN       -0.611574
GOOGL      -0.741701
MSFT     